<a href="https://colab.research.google.com/github/francescopatane96/Computer_aided_drug_discovery_kit/blob/main/ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this module, you will learn more about the ChEMBL database and how to extract data from it for a target of interest. Data sets can be used for many cheminformatics tasks, eg. similarity search and clustering or machine learning.

In this notebook you will find compounds which were tested against a specific target and filtering available bioactivity data.

In [ ]:
! pip install chembl_webresource_client    

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd
import math
import rdkit
from tqdm.auto import tqdm
from chembl_webresource_client.new_client import new_client
from pandas import DataFrame
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, PandasTools
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory

In [ ]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

create resource objects for API access

In [ ]:
targets_api = new_client.target
compounds_api = new_client.molecule
bioactivities_api = new_client.activity

In [ ]:
type(targets_api)  #show the type of the object

In [ ]:
uniprot_id = "P00533"    #change the uniprot ID for your project

Fetch target data from ChEMBL

In [ ]:
# Get target information from ChEMBL but restrict it to specified class only
targets = targets_api.get(target_components__accession=uniprot_id).only(              ##variable that contains the results of the query
    "target_chembl_id", "organism", "pref_name", "target_type"
)
print(f'The type of the targets is "{type(targets)}"')

Download target data from ChEMBL

In [ ]:
targets = pd.DataFrame(targets)
targets

Select target (ChEMBL ID)

In [ ]:
target = targets.iloc[0]
target

save chembl id

In [ ]:
target_id = target.target_chembl_id
print(f"The target ChEMBL ID is {target_id}")

Get Bioactivity data

In [ ]:
bioactivities = bioactivities_api.filter(
    target_chembl_id=target_id, type="IC50", relation="=", assay_type="B"
).only(
    "activity_id",
    "assay_chembl_id",
    "assay_description",
    "assay_type",
    "molecule_chembl_id",
    "type",
    "standard_units",
    "relation",
    "standard_value",
    "target_chembl_id",
    "target_organism",
)

print(f"Length and type of bioactivities object: {len(bioactivities)}, {type(bioactivities)}")

In [ ]:
print(f"Length and type of first element: {len(bioactivities[0])}, {type(bioactivities[0])}")
bioactivities[0]

Download Bioactivity data from ChEMBL

In [ ]:
bioactivities_df = pd.DataFrame.from_records(bioactivities)
print(f"DataFrame shape: {bioactivities_df.shape}")
bioactivities_df.head()

convert values to nM

In [ ]:
bioactivities_df['units'].unique()

In [ ]:
bioactivities_df.drop(["units", "value"], axis=1, inplace=True)



In [ ]:
bioactivities_df.head()

Preprocess and filter bioactivity data

1. Convert datatype of “standard_value” from “object” to “float”


In [ ]:
bioactivities_df.dtypes

In [ ]:
bioactivities_df = bioactivities_df.astype({"standard_value" : "float64"})

In [ ]:
bioactivities_df.dtypes

2. Delete entries with missing values

In [ ]:
bioactivities_df.dropna(axis=0, how="any", inplace=True)   #drop rows which contain missing values
print(f"DataFrame shape: {bioactivities_df.shape}")

3. Keep only entries with “standard_unit == nM”

In [ ]:
print(f"Units in downloaded data: {bioactivities_df['standard_units'].unique()}")
print(
    f"Number of non-nM entries:\
    {bioactivities_df[bioactivities_df['standard_units'] != 'nM'].shape[0]}"
)

In [ ]:
bioactivities_df = bioactivities_df[bioactivities_df["standard_units"] == "nM"]
print(f"Units after filtering: {bioactivities_df['standard_units'].unique()}")

In [ ]:
print(f"DataFrame shape: {bioactivities_df.shape}")

4. Delete duplicate molecules

In [ ]:
bioactivities_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"DataFrame shape: {bioactivities_df.shape}")

5. Reset “DataFrame” index

In [ ]:
bioactivities_df.reset_index(drop=True, inplace=True)
bioactivities_df.head()


6. Rename columns

In [ ]:
bioactivities_df.rename(
    columns={"standard_value": "IC50", "standard_units": "units"}, inplace=True
)
bioactivities_df.head()

In [ ]:
print(f"DataFrame shape: {bioactivities_df.shape}")

Fetch compound data from ChEMBL

In [ ]:
compounds_provider = compounds_api.filter(
    molecule_chembl_id__in=list(bioactivities_df["molecule_chembl_id"])
).only("molecule_chembl_id", "molecule_structures")

Download compound data from ChEMBL

In [ ]:
compounds = list(tqdm(compounds_provider))

In [ ]:
compounds_df = pd.DataFrame.from_records(
    compounds,
)
print(f"DataFrame shape: {compounds_df.shape}")

In [ ]:
compounds_df.head()

Preprocess and filter compound data

1. Remove entries with missing molecule structure entry

In [ ]:
compounds_df.dropna(axis=0, how="any", inplace=True)
print(f"DataFrame shape: {compounds_df.shape}")

2. Delete duplicate molecules

In [ ]:
compounds_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"DataFrame shape: {compounds_df.shape}")

3. Get molecules with canonical SMILES

In [ ]:
compounds_df.iloc[0].molecule_structures.keys()

In [ ]:
canonical_smiles = []

for i, compounds in compounds_df.iterrows():
    try:
        canonical_smiles.append(compounds["molecule_structures"]["canonical_smiles"])
    except KeyError:
        canonical_smiles.append(None)

compounds_df["smiles"] = canonical_smiles
compounds_df.drop("molecule_structures", axis=1, inplace=True)
print(f"DataFrame shape: {compounds_df.shape}")

In [ ]:
compounds_df.dropna(axis=0, how="any", inplace=True)
print(f"DataFrame shape: {compounds_df.shape}")

Summary of compound and bioactivity data

In [ ]:
print(f"Bioactivities filtered: {bioactivities_df.shape[0]}")
bioactivities_df.columns

In [ ]:
print(f"Compounds filtered: {compounds_df.shape[0]}")
compounds_df.columns

Merge both datasets

In [ ]:
# Merge DataFrames
output_df = pd.merge(
    bioactivities_df[["molecule_chembl_id", "IC50", "units"]],
    compounds_df,
    on="molecule_chembl_id",
)

# Reset row indices
output_df.reset_index(drop=True, inplace=True)

print(f"Dataset with {output_df.shape[0]} entries.")

In [ ]:
output_df.dtypes

In [ ]:
output_df.head(10)

Add pIC50 values

In [ ]:
def convert_ic50_to_pic50(IC50_value):
    pIC50_value = 9 - math.log10(IC50_value)
    return pIC50_value

In [ ]:
# Apply conversion to each row of the compounds DataFrame
output_df["pIC50"] = output_df.apply(lambda x: convert_ic50_to_pic50(x.IC50), axis=1)

In [ ]:
output_df.head()

Draw compound data

In [ ]:
output_df.hist(column="pIC50")

In [ ]:
# Add molecule column
PandasTools.AddMoleculeColumnToFrame(output_df, smilesCol="smiles")

In [ ]:
# Sort molecules by pIC50
output_df.sort_values(by="pIC50", ascending=False, inplace=True)

In [ ]:
# Reset index
output_df.reset_index(drop=True, inplace=True)

In [ ]:
output_df.drop("smiles", axis=1).head(10)

In [ ]:
print(f"DataFrame shape: {output_df.shape}")

In [ ]:
output_df.to_csv("EGFR_compounds.csv")
output_df.head()

da qui 

In [ ]:
target = new_client.target                                     
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
print(targets)

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
print(res)

In [ ]:
df = pd.DataFrame.from_dict(res)
print(df)

In [ ]:
df.to_csv('acetylcholinesterase_bioactivity_data.csv', index=False)

In [ ]:
act_normal = df[df.standard_value.notna()]
act_normal = act_normal[act_normal.canonical_smiles.notna()]
act_normal = act_normal.drop_duplicates(['canonical_smiles'])
act_normal

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
new_act = act_normal[selection]
new_act

In [ ]:
new_act.to_csv('acetylcholinesterase_bioactivity_clear', index=False)

In [ ]:
# This is temporary line
new_act = pd.read_csv('acetylcholinesterase_bioactivity_clear')

In [ ]:
bioactivity_threshold = []
for i in new_act.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append('inactive')
  elif float(i) <= 1000:
    bioactivity_threshold.append('active')
  else:
    bioactivity_threshold.append('intermediate')
bioactivity_class = pd.Series(bioactivity_threshold, name = 'bioactivity_class')
act5 = pd.concat([new_act, bioactivity_class], axis=1)
act5

In [ ]:
act5 = act5.dropna()
act5

In [ ]:
act5.to_csv('bioactivity_with_class.csv', index=False)

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
act_nosmiles = act5.drop(columns='canonical_smiles')
smiles = []

for i in act5.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name='canonical_smiles')
act_clean_smiles = pd.concat([act_nosmiles,smiles], axis=1)
act_clean_smiles

In [ ]:

def lipinski(smiles, verbose=False):
    moldata = []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
    baseData = np.arange(1, 1)
    i = 0
    for mol in moldata:

        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
  
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])
        if (i==0):
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i = i + 1

    columnNames =  ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)
    return descriptors


In [ ]:
act_lipinski = lipinski(act_clean_smiles.canonical_smiles)
act_lipinski

In [ ]:
act_comb = pd.concat([act5, act_lipinski], axis = 1)
act_comb

normalizing standard values

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [ ]:
act_norm = norm_value(act_comb)
act_norm

converting to pIC50

In [ ]:
def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [ ]:
act_final = pIC50(act_norm)
act_final

In [ ]:
act_final.to_csv('bioactvity_pIC50.csv')

Exploratory data analysis (chemical space analysis) via lipinski descriptors

In [ ]:
act_fn = act_final[act_final['bioactivity_class'] != 'intermediate']
act_fn

frequency of 2 classes (active, inactive)

In [ ]:
plt.figure(figsize=(5.5, 5.5))
sns.countplot(x='bioactivity_class', data=act_fn, edgecolor='black')
plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
plt.savefig('plot_bioactivity_class.pdf')

making statistical analysis

In [ ]:
def mannwhitney(descriptor, verbose=False):
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'bioactivity_class']
  df = act_fn[selection]
  active = df[df['bioactivity_class'] == 'active']
  active = active[descriptor]

  selection = [descriptor, 'bioactivity_class']
  df = act_fn[selection]
  inactive = df[df['bioactivity_class'] == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'
  
  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'pIC50', data = act_fn)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

In [ ]:
mannwhitney('pIC50')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'MW', data = act_fn)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW value', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

In [ ]:
mannwhitney('MW')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=act_fn, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'LogP', data = act_fn)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP value', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

In [ ]:
mannwhitney('LogP')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHDonors', data = act_fn)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDon value', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDon.pdf')

In [ ]:
mannwhitney('NumHDonors')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHAcceptors', data = act_fn)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcc value', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHAcc.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

results interpretation

All of the four Lipinski descriptors exhibited statistically significant difference between active and inactive molecules.

let's calculate other descriptors with PADEL

In [ ]:
! wget https://github.com/gromdimon/features/raw/main/padel.sh
! wget https://github.com/gromdimon/features/raw/main/padel.zip

In [ ]:
!unzip padel.zip

In [ ]:
act_final

In [ ]:
selection = ['canonical_smiles', 'molecule_chembl_id']
act_selected = act_final[selection]
act_selected.to_csv('molecule.smi', sep='\t', index=False, header=False )

In [ ]:
! cat molecule.smi | head -5
! cat molecule.smi | wc -l

In [ ]:
!cat padel.sh

In [ ]:
!bash padel.sh

In [ ]:
!ls -l

preparing data for later researchs

In [ ]:
actx = pd.read_csv('descriptors_output.csv')
actx

In [ ]:
actx = actx.drop(columns='Name')
actx


In [ ]:
actx.to_csv('actx.csv')

In [ ]:
acty = act_final['pIC50']
acty

In [ ]:
actx.shape

In [ ]:
acty.shape

making new datase

In [ ]:
datasetxy = pd.concat([actx, acty], axis=1)
datasetxy

In [ ]:
datasetxy.to_csv('dataset_with_padel_pIC50.csv', index=False)